In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)4
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)


# OUR RANGE: SENSEX (15-3-2008 to 15-3-2024)

In [ ]:
vix = pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/India VIX Historical Data (3).csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes
data=pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/BSE Sensex 30 Historical Data (3).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# Separate features (X) and target (y)
y = data[['Close']].copy()
X = data.drop('Close', axis=1).copy()

# It's important to scale your features and target separately.
# We'll need the 'scaler_y' later to inverse the transformation of our predictions.
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)


# ---------------------------
# 3. Create 15-day Sequences for Multivariate Data
# ---------------------------
def create_multivariate_sequences(X_data, y_data, time_steps=15):
    """
    Creates sequences for multivariate time series data.
    """
    Xs, ys = [], []
    for i in range(len(X_data) - time_steps):
        # Get a sequence of 'time_steps' from the features
        Xs.append(X_data[i:(i + time_steps)])
        # The target is the 'Close' price at the end of the sequence
        ys.append(y_data[i + time_steps])
    return np.array(Xs), np.array(ys)


# Create the sequences
X_seq, y_seq = create_multivariate_sequences(X_scaled, y_scaled, time_steps=15)

# Split the data into training, validation, and testing sets
train_size = int(len(X_seq) * 0.7)
val_size = int(len(X_seq) * 0.1)

X_train, y_train = X_seq[:train_size], y_seq[:train_size]
X_val, y_val = X_seq[train_size:train_size + val_size], y_seq[train_size:train_size + val_size]
X_test, y_test = X_seq[train_size + val_size:], y_seq[train_size + val_size:]

# The input for the model will have a shape of (number of samples, timesteps, number of features)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
n_features = X.shape[1]

## with single LSTM model

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_lstm_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))
    # Simple LSTM (unidirectional)
    x = LSTM(128, return_sequences=False, dropout=0.2)(inputs)
    x = Dense(64, activation='relu')(x)
    x = Dense(1, activation='tanh')(x)
    x = Lambda(lambda x: x * scale_factor)(x)

    # Use the last day's price for final output calculation
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, x])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_lstm_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## 15‑Day Branch with Bidirectional LSTM

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm1_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))
    # 15-day full branch with BiLSTM
    x = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    x = Dense(64, activation='relu')(x)
    x = Dense(1, activation='tanh')(x)
    x = Lambda(lambda x: x * scale_factor)(x)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, x])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm1_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## 15‑Day + 7‑Day Branch

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm2_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Recent branch (last 7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Gate the 7-day branch output
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])

    # Fuse the two branches
    fused_returns = Add()([long_term_return, gated_mid_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm2_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## 7‑Day + 1‑Day Branch

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm3_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Recent branch (last 7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Very recent branch (last 1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gating mechanisms
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Combine gated outputs
    fused_returns = Add()([gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm3_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## 15‑Day + 1‑Day Branch

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm4_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Very recent branch (last 1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gate the 1-day branch output
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Fuse branches
    fused_returns = Add()([long_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm4_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## Full Model – 15‑Day + 7‑Day + 1‑Day Branch

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm5_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Recent branch (7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Very recent branch (1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gating mechanisms
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Fuse all branches
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm5_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


# OUR RANGE: NIFTY50 (15-3-2008 to 15-3-2024)

In [ ]:
vix = pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/India VIX Historical Data (3).csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes
data=pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/Nifty 50 Historical Data (2).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

#data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# Separate features (X) and target (y)
data.dropna(axis=0,inplace=True)
y = data[['Close']].copy()
X = data.drop('Close', axis=1).copy()

# It's important to scale your features and target separately.
# We'll need the 'scaler_y' later to inverse the transformation of our predictions.
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)


# ---------------------------
# 3. Create 15-day Sequences for Multivariate Data
# ---------------------------
def create_multivariate_sequences(X_data, y_data, time_steps=15):
    """
    Creates sequences for multivariate time series data.
    """
    Xs, ys = [], []
    for i in range(len(X_data) - time_steps):
        # Get a sequence of 'time_steps' from the features
        Xs.append(X_data[i:(i + time_steps)])
        # The target is the 'Close' price at the end of the sequence
        ys.append(y_data[i + time_steps])
    return np.array(Xs), np.array(ys)


# Create the sequences
X_seq, y_seq = create_multivariate_sequences(X_scaled, y_scaled, time_steps=15)

# Split the data into training, validation, and testing sets
train_size = int(len(X_seq) * 0.7)
val_size = int(len(X_seq) * 0.1)

X_train, y_train = X_seq[:train_size], y_seq[:train_size]
X_val, y_val = X_seq[train_size:train_size + val_size], y_seq[train_size:train_size + val_size]
X_test, y_test = X_seq[train_size + val_size:], y_seq[train_size + val_size:]

# The input for the model will have a shape of (number of samples, timesteps, number of features)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
n_features = X.shape[1]

## with single LSTM

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_lstm_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))
    # Simple LSTM (unidirectional)
    x = LSTM(128, return_sequences=False, dropout=0.2)(inputs)
    x = Dense(16)(x)
    x = Dense(1,activation="tanh")(x)
    x = Lambda(lambda x: x * scale_factor)(x)

    # Use the last day's price for final output calculation
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, x])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_lstm_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
vix = pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/India VIX Historical Data (3).csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes
data=pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/Nifty 50 Historical Data (2).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# Separate features (X) and target (y)
data.dropna(axis=0,inplace=True)
y = data[['Close']].copy()
X = data.drop('Close', axis=1).copy()

# It's important to scale your features and target separately.
# We'll need the 'scaler_y' later to inverse the transformation of our predictions.
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)


# ---------------------------
# 3. Create 15-day Sequences for Multivariate Data
# ---------------------------
def create_multivariate_sequences(X_data, y_data, time_steps=15):
    """
    Creates sequences for multivariate time series data.
    """
    Xs, ys = [], []
    for i in range(len(X_data) - time_steps):
        # Get a sequence of 'time_steps' from the features
        Xs.append(X_data[i:(i + time_steps)])
        # The target is the 'Close' price at the end of the sequence
        ys.append(y_data[i + time_steps])
    return np.array(Xs), np.array(ys)


# Create the sequences
X_seq, y_seq = create_multivariate_sequences(X_scaled, y_scaled, time_steps=15)

# Split the data into training, validation, and testing sets
train_size = int(len(X_seq) * 0.7)
val_size = int(len(X_seq) * 0.1)

X_train, y_train = X_seq[:train_size], y_seq[:train_size]
X_val, y_val = X_seq[train_size:train_size + val_size], y_seq[train_size:train_size + val_size]
X_test, y_test = X_seq[train_size + val_size:], y_seq[train_size + val_size:]

# The input for the model will have a shape of (number of samples, timesteps, number of features)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
n_features = X.shape[1]

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_lstm_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))
    # Simple LSTM (unidirectional)
    x = LSTM(128, return_sequences=False, dropout=0.2)(inputs)
    x = Dense(16)(x)
    x = Dense(1,activation="tanh")(x)
    x = Lambda(lambda x: x * scale_factor)(x)

    # Use the last day's price for final output calculation
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, x])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_lstm_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## 15-Day Branch with Bidirectional LSTM

In [ ]:
from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm1_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))
    # 15-day full branch with BiLSTM
    x = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    x = Dense(64, activation='relu')(x)
    x = Dense(1, activation='tanh')(x)
    x = Lambda(lambda x: x * scale_factor)(x)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, x])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm1_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## 15-Day + 7-Day Branch

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm2_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Recent branch (last 7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Gate the 7-day branch output
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])

    # Fuse the two branches
    fused_returns = Add()([long_term_return, gated_mid_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm2_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## 7-Day + 1-Day Branch

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm3_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Recent branch (last 7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Very recent branch (last 1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gating mechanisms
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Combine gated outputs
    fused_returns = Add()([gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm3_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## 15-Day + 1-Day Branch

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm4_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Very recent branch (last 1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gate the 1-day branch output
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Fuse branches
    fused_returns = Add()([long_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm4_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## Full Model (15‑Day + 7‑Day + 1‑Day Branch)

In [ ]:
vix = pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/India VIX Historical Data (3).csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes
data=pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/Nifty 50 Historical Data (2).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# Separate features (X) and target (y)
data.dropna(axis=0,inplace=True)
y = data[['Close']].copy()
X = data.drop('Close', axis=1).copy()

# It's important to scale your features and target separately.
# We'll need the 'scaler_y' later to inverse the transformation of our predictions.
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)


# ---------------------------
# 3. Create 15-day Sequences for Multivariate Data
# ---------------------------
def create_multivariate_sequences(X_data, y_data, time_steps=15):
    """
    Creates sequences for multivariate time series data.
    """
    Xs, ys = [], []
    for i in range(len(X_data) - time_steps):
        # Get a sequence of 'time_steps' from the features
        Xs.append(X_data[i:(i + time_steps)])
        # The target is the 'Close' price at the end of the sequence
        ys.append(y_data[i + time_steps])
    return np.array(Xs), np.array(ys)


# Create the sequences
X_seq, y_seq = create_multivariate_sequences(X_scaled, y_scaled, time_steps=15)

# Split the data into training, validation, and testing sets
train_size = int(len(X_seq) * 0.7)
val_size = int(len(X_seq) * 0.1)

X_train, y_train = X_seq[:train_size], y_seq[:train_size]
X_val, y_val = X_seq[train_size:train_size + val_size], y_seq[train_size:train_size + val_size]
X_test, y_test = X_seq[train_size + val_size:], y_seq[train_size + val_size:]

# The input for the model will have a shape of (number of samples, timesteps, number of features)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
n_features = X.shape[1]

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm5_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Recent branch (7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Very recent branch (1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gating mechanisms
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Fuse all branches
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm5_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
vix = pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/India VIX Historical Data (3).csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes
data=pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/Nifty 50 Historical Data (2).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

#data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# Separate features (X) and target (y)
data.dropna(axis=0,inplace=True)
y = data[['Close']].copy()
X = data.drop('Close', axis=1).copy()

# It's important to scale your features and target separately.
# We'll need the 'scaler_y' later to inverse the transformation of our predictions.
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)


# ---------------------------
# 3. Create 15-day Sequences for Multivariate Data
# ---------------------------
def create_multivariate_sequences(X_data, y_data, time_steps=15):
    """
    Creates sequences for multivariate time series data.
    """
    Xs, ys = [], []
    for i in range(len(X_data) - time_steps):
        # Get a sequence of 'time_steps' from the features
        Xs.append(X_data[i:(i + time_steps)])
        # The target is the 'Close' price at the end of the sequence
        ys.append(y_data[i + time_steps])
    return np.array(Xs), np.array(ys)


# Create the sequences
X_seq, y_seq = create_multivariate_sequences(X_scaled, y_scaled, time_steps=15)

# Split the data into training, validation, and testing sets
train_size = int(len(X_seq) * 0.7)
val_size = int(len(X_seq) * 0.1)

X_train, y_train = X_seq[:train_size], y_seq[:train_size]
X_val, y_val = X_seq[train_size:train_size + val_size], y_seq[train_size:train_size + val_size]
X_test, y_test = X_seq[train_size + val_size:], y_seq[train_size + val_size:]

# The input for the model will have a shape of (number of samples, timesteps, number of features)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
n_features = X.shape[1]
      

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm5_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Recent branch (7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Very recent branch (1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gating mechanisms
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Fuse all branches
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm5_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


## OUR RANGE:SP500 (15-3-2008 to 15-3-2024)

- with single lstm

- - without vix

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/kaggle/input/sp500-data/SP500 Vix (1).csv")

# Ensure the required columns exist
if "Price" in df.columns and "Open" in df.columns:
    # Calculate the percentage change
    df["Change %"] = ((df["Price"] - df["Open"]) / df["Price"]) * 100

    # Save the updated CSV file
    # df.to_csv("SP 500 (2).csv", index=False)

    # Display first few rows
    print(df.head())
else:
    print("Error: CSV file must contain 'Price' and 'Open' columns.")
df.head()

In [ ]:
import pandas as pd
import numpy as np
vix = df
vix.drop("Vol.",axis=1,inplace=True)
def convert_volume(val):
    if pd.isna(val) or val == '-':  # Handle NaN or '-' cases
        return np.nan
    val = val.strip().replace(',', '')  # Remove spaces & commas
    if val[-1] == 'K': return float(val[:-1]) * 1e3
    if val[-1] == 'M': return float(val[:-1]) * 1e6
    if val[-1] == 'B': return float(val[:-1]) * 1e9
    return float(val)

#vix["Vol."] = vix["Vol."].apply(convert_volume)
vix.rename(columns={'Open':'Vopen','High':'Vhigh','Low':'Vlow','Change %':'Vchange%'}, inplace=True)
vix["Vchange%"] = vix["Vchange%"].astype(str).str.replace("%", "").astype(float)
vix.head()

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/kaggle/input/sp500-data/sp500_historical_data (2).csv")

# Ensure the required columns exist
if "Price" in df.columns and "Open" in df.columns:
    # Calculate the percentage change
    df["Change %"] = ((df["Price"] - df["Open"]) / df["Price"]) * 100

    # Save the updated CSV file
    # df.to_csv("SP 500 (2).csv", index=False)

    # Display first few rows
    print(df.head())
else:
    print("Error: CSV file must contain 'Price' and 'Open' columns.")
df.head()

In [ ]:
data=df
data=data[::-1]
data.reset_index(drop=True, inplace=True)
print(data.head())
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
#df.drop("Vol.",axis=1,inplace=True)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df
#data['Date'] = pd.to_datetime(data['Date'], format="%d-%m-%Y")

# For 'vix', the date format is "MM/DD/YYYY"
#vix['Date'] = pd.to_datetime(vix['Date'], format="%m/%d/%Y")
#data = pd.merge(data, vix, on='Date', how='inner')
#row=data[data["Vol."].isna()]
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# Separate features (X) and target (y)
y = data[['Close']].copy()
X = data.drop('Close', axis=1).copy()

# It's important to scale your features and target separately.
# We'll need the 'scaler_y' later to inverse the transformation of our predictions.
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)


# ---------------------------
# 3. Create 15-day Sequences for Multivariate Data
# ---------------------------
def create_multivariate_sequences(X_data, y_data, time_steps=15):
    """
    Creates sequences for multivariate time series data.
    """
    Xs, ys = [], []
    for i in range(len(X_data) - time_steps):
        # Get a sequence of 'time_steps' from the features
        Xs.append(X_data[i:(i + time_steps)])
        # The target is the 'Close' price at the end of the sequence
        ys.append(y_data[i + time_steps])
    return np.array(Xs), np.array(ys)


# Create the sequences
X_seq, y_seq = create_multivariate_sequences(X_scaled, y_scaled, time_steps=15)

# Split the data into training, validation, and testing sets
train_size = int(len(X_seq) * 0.7)
val_size = int(len(X_seq) * 0.1)

X_train, y_train = X_seq[:train_size], y_seq[:train_size]
X_val, y_val = X_seq[train_size:train_size + val_size], y_seq[train_size:train_size + val_size]
X_test, y_test = X_seq[train_size + val_size:], y_seq[train_size + val_size:]

# The input for the model will have a shape of (number of samples, timesteps, number of features)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
n_features = X.shape[1]

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_lstm_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))
    # Simple LSTM (unidirectional)
    x = LSTM(128, return_sequences=False, dropout=0.2)(inputs)
    x = Dense(16)(x)
    x = Dense(1,activation="tanh")(x)
    x = Lambda(lambda x: x * scale_factor)(x)

    # Use the last day's price for final output calculation
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, x])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_lstm_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility


plt.figure(figsize=(12, 6))

num_points = max(100, len(y_test_inv))

# Actual: Royal Blue
plt.plot(
    y_test_inv[:num_points],
    label="Actual",
    linestyle="-",
    linewidth=3.5,
    color="#4169E1"  # Royal Blue
)

# Predicted: Crimson Red
plt.plot(
    y_pred_inv[:num_points],
    label="Predicted",
    linestyle="--",
    linewidth=3.5,
    color="#DC143C"  # Crimson Red
)

plt.xlabel("Time", fontsize=14, fontweight="bold")
plt.ylabel("Stock Price", fontsize=14, fontweight="bold")
plt.title("Actual vs Predicted Stock Prices", fontsize=16, fontweight="bold")

plt.legend(fontsize=12, loc="upper left")
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()


In [ ]:
from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm1_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))
    # 15-day full branch with BiLSTM
    x = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    x = Dense(64, activation='relu')(x)
    x = Dense(1, activation='tanh')(x)
    x = Lambda(lambda x: x * scale_factor)(x)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, x])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm1_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
plt.figure(figsize=(12, 6))

num_points = max(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8, color="#1f77b4")
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8, color="#ff7f0e")

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.figure(figsize=(12, 6))

num_points = max(100, len(y_test_inv))

# Actual: Royal Blue
plt.plot(
    y_test_inv[:num_points],
    label="Actual",
    linestyle="-",
    linewidth=3.5,
    color="#4169E1"  # Royal Blue
)

# Predicted: Crimson Red
plt.plot(
    y_pred_inv[:num_points],
    label="Predicted",
    linestyle="--",
    linewidth=3.5,
    color="#DC143C"  # Crimson Red
)

plt.xlabel("Time", fontsize=14, fontweight="bold")
plt.ylabel("Stock Price", fontsize=14, fontweight="bold")
plt.title("Actual vs Predicted Stock Prices", fontsize=16, fontweight="bold")

plt.legend(fontsize=12, loc="upper left")
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()


In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm2_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Recent branch (last 7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Gate the 7-day branch output
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])

    # Fuse the two branches
    fused_returns = Add()([long_term_return, gated_mid_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm2_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm3_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Recent branch (last 7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Very recent branch (last 1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gating mechanisms
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Combine gated outputs
    fused_returns = Add()([gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm3_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm5_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Recent branch (7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Very recent branch (1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gating mechanisms
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Fuse all branches
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm5_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


- with vix

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/kaggle/input/sp500-data/SP500 Vix (1).csv")

# Ensure the required columns exist
if "Price" in df.columns and "Open" in df.columns:
    # Calculate the percentage change
    df["Change %"] = ((df["Price"] - df["Open"]) / df["Price"]) * 100

    # Save the updated CSV file
    # df.to_csv("SP 500 (2).csv", index=False)

    # Display first few rows
    print(df.head())
else:
    print("Error: CSV file must contain 'Price' and 'Open' columns.")
df.head()

In [ ]:
import pandas as pd
import numpy as np
vix = df
vix.drop("Vol.",axis=1,inplace=True)
def convert_volume(val):
    if pd.isna(val) or val == '-':  # Handle NaN or '-' cases
        return np.nan
    val = val.strip().replace(',', '')  # Remove spaces & commas
    if val[-1] == 'K': return float(val[:-1]) * 1e3
    if val[-1] == 'M': return float(val[:-1]) * 1e6
    if val[-1] == 'B': return float(val[:-1]) * 1e9
    return float(val)

#vix["Vol."] = vix["Vol."].apply(convert_volume)
vix.rename(columns={'Open':'Vopen','High':'Vhigh','Low':'Vlow','Change %':'Vchange%'}, inplace=True)
vix["Vchange%"] = vix["Vchange%"].astype(str).str.replace("%", "").astype(float)
vix.head()

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/kaggle/input/sp500-data/sp500_historical_data (2).csv")

# Ensure the required columns exist
if "Price" in df.columns and "Open" in df.columns:
    # Calculate the percentage change
    df["Change %"] = ((df["Price"] - df["Open"]) / df["Price"]) * 100

    # Save the updated CSV file
    # df.to_csv("SP 500 (2).csv", index=False)

    # Display first few rows
    print(df.head())
else:
    print("Error: CSV file must contain 'Price' and 'Open' columns.")
df.head()

In [ ]:
data=df
data=data[::-1]
data.reset_index(drop=True, inplace=True)
print(data.head())
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
#df.drop("Vol.",axis=1,inplace=True)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df
#data['Date'] = pd.to_datetime(data['Date'], format="%d-%m-%Y")

# For 'vix', the date format is "MM/DD/YYYY"
#vix['Date'] = pd.to_datetime(vix['Date'], format="%m/%d/%Y")
data = pd.merge(data, vix, on='Date', how='inner')
#row=data[data["Vol."].isna()]
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# Separate features (X) and target (y)
y = data[['Close']].copy()
X = data.drop('Close', axis=1).copy()

# It's important to scale your features and target separately.
# We'll need the 'scaler_y' later to inverse the transformation of our predictions.
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)


# ---------------------------
# 3. Create 15-day Sequences for Multivariate Data
# ---------------------------
def create_multivariate_sequences(X_data, y_data, time_steps=15):
    """
    Creates sequences for multivariate time series data.
    """
    Xs, ys = [], []
    for i in range(len(X_data) - time_steps):
        # Get a sequence of 'time_steps' from the features
        Xs.append(X_data[i:(i + time_steps)])
        # The target is the 'Close' price at the end of the sequence
        ys.append(y_data[i + time_steps])
    return np.array(Xs), np.array(ys)


# Create the sequences
X_seq, y_seq = create_multivariate_sequences(X_scaled, y_scaled, time_steps=15)

# Split the data into training, validation, and testing sets
train_size = int(len(X_seq) * 0.7)
val_size = int(len(X_seq) * 0.1)

X_train, y_train = X_seq[:train_size], y_seq[:train_size]
X_val, y_val = X_seq[train_size:train_size + val_size], y_seq[train_size:train_size + val_size]
X_test, y_test = X_seq[train_size + val_size:], y_seq[train_size + val_size:]

# The input for the model will have a shape of (number of samples, timesteps, number of features)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
n_features = X.shape[1]

In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_lstm_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))
    # Simple LSTM (unidirectional)
    x = LSTM(128, return_sequences=False, dropout=0.2)(inputs)
    x = Dense(16)(x)
    x = Dense(1,activation="tanh")(x)
    x = Lambda(lambda x: x * scale_factor)(x)

    # Use the last day's price for final output calculation
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, x])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_lstm_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm1_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))
    # 15-day full branch with BiLSTM
    x = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    x = Dense(64, activation='relu')(x)
    x = Dense(1, activation='tanh')(x)
    x = Lambda(lambda x: x * scale_factor)(x)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, x])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm1_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm2_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Recent branch (last 7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Gate the 7-day branch output
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])

    # Fuse the two branches
    fused_returns = Add()([long_term_return, gated_mid_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm2_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm3_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Recent branch (last 7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Very recent branch (last 1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gating mechanisms
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Combine gated outputs
    fused_returns = Add()([gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm3_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()


In [ ]:

from tensorflow.keras.layers import Multiply, Activation

def build_Bilstm5_model(seq_length=15,n_features=n_features, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, n_features))

    # Full branch (15 days)
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # Recent branch (7 days)
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # Very recent branch (1 day)
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # Gating mechanisms
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)
    gated_mid_term_return = Multiply()([mid_term_return, gate_7])
    gated_short_term_return = Multiply()([short_term_return, gate_1])

    # Fuse all branches
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_Bilstm5_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv)

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")
plt.figure(figsize=(12, 6))

# Plot the first 100 points for better visibility
num_points = max(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices (Zoomed In)")
plt.legend()
plt.grid(True)

plt.show()

plt.figure(figsize=(12, 6))

num_points = max(100, len(y_test_inv))

# Actual: Royal Blue
plt.plot(
    y_test_inv[:num_points],
    label="Actual",
    linestyle="-",
    linewidth=3.5,
    color="#4169E1"  # Royal Blue
)

# Predicted: Crimson Red
plt.plot(
    y_pred_inv[:num_points],
    label="Predicted",
    linestyle="--",
    linewidth=3.5,
    color="#DC143C"  # Crimson Red
)

plt.xlabel("Time", fontsize=14, fontweight="bold")
plt.ylabel("Stock Price", fontsize=14, fontweight="bold")
plt.title("Actual vs Predicted Stock Prices", fontsize=16, fontweight="bold")

plt.legend(fontsize=12, loc="upper left")
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()


In [ ]:
plt.figure(figsize=(12, 6))

num_points = max(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices")
plt.legend()
plt.grid(True)

plt.show()

plt.figure(figsize=(12, 6))

num_points = max(100, len(y_test_inv))

# Actual: Royal Blue
plt.plot(
    y_test_inv[:num_points],
    label="Actual",
    linestyle="-",
    linewidth=3.5,
    color="#4169E1"  # Royal Blue
)

# Predicted: Crimson Red
plt.plot(
    y_pred_inv[:num_points],
    label="Predicted",
    linestyle="--",
    linewidth=3.5,
    color="#DC143C"  # Crimson Red
)

plt.xlabel("Time", fontsize=14, fontweight="bold")
plt.ylabel("Stock Price", fontsize=14, fontweight="bold")
plt.title("Actual vs Predicted Stock Prices", fontsize=16, fontweight="bold")

plt.legend(fontsize=12, loc="upper left")
plt.grid(True, linestyle="--", alpha=0.5)

plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
num_points = min(100, len(y_test_inv))

plt.plot(y_test_inv[:num_points], label="Actual", linestyle="-", linewidth=2, marker="o", markersize=3, alpha=0.8)
plt.plot(y_pred_inv[:num_points], label="Predicted", linestyle="--", linewidth=2, marker="x", markersize=3, alpha=0.8)

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Actual vs Predicted Stock Prices")
plt.legend()
plt.grid(True)

plt.show()
